# Is There A Correlation Between County and Number of Critical Deficiencies Found per Sanitary Inspection?


![picture](https://github.com/andrewcsmith1997/retailFoodStoreCountyAnalysis/blob/master/Images/healthgrade.png)

To answer that question, I conducted two sample t-tests comparing the mean (number of critical deficiencies found per inspection) of each county to the mean of every other county combined.

## Original Data Set

I found the data that raised this question on [data.ny.gov](https://data.ny.gov/Economic-Development/Retail-Food-Store-Inspections-Current-Ratings/d6dy-3h7r); it was posted by the Department of Agriculture and Markets, Division of Food Safety and Inspection. I downloaded the CSV and imported it into a table in my personal MySQL database.


In [ ]:
!pip install mysql-connector-python

In [ ]:
# Imported libraries
import mysql.connector
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from scipy.stats import norm


In [ ]:
# Establishing the connection to the mySQL database
mySQLuser = 'removed for privacy'
mySQLpasswd = 'removed for privacy'
mySQLhost = 'removed for privacy'
mySQLport = 'removed for privacy'
mySQLdatabase = 'removed for privacy'
connection = mysql.connector.connect(host=mySQLhost, port = mySQLport, 
                                     user=mySQLuser, passwd=mySQLpasswd, 
                                     db=mySQLdatabase, 
                                     auth_plugin='mysql_native_password')
# Adjusts column width for data frame to fit more characters if necessary
pd.set_option('max_colwidth', 300)

The data set consists of roughly 93.3 thousand records from 03*/*01/2019 to 02*/*29/2020. Each record is either a deficiency found during an inspection or an entry stating that there were no deficiencies found.

In [ ]:
# Creates Data Frame from SQL Query
allInfoQuery = '''
    SELECT *
    FROM retail_food_store_inspections;
'''
allInfoDF = pd.read_sql_query(allInfoQuery, con = connection)
allInfoDF

,county,inspection,inspection_date,owner_name,trade_name,street,city,state_code,zip_code,deficiency_number,deficiency_description,location_1
0,Albany,C,2019-08-21,ALBANY FAST FOOD INC,ALBANY FAST FOOD,9 NEW SCOTLAND AVE,ALBANY,NY,12208,02A,"- 15.46 pounds of bagged chips, bagged flour, and packages of noodles stored on shelves in retail area found to be rodent defiled with gnaw marks. Products destroyed under signed waiver during inspection.","9 NEW SCOTLAND AVE\nALBANY, NY 12208"
1,Albany,A,2019-10-09,ALL STAR PIZZA & DELI INC,ALL STAR PIZZA & DELI,224 QUAIL STREET,ALBANY,NY,12203,None,None,"224 QUAIL STREET\nALBANY, NY 12203"
2,Albany,A,2019-09-06,EASHA INC,EASHA MART,206 QUAIL ST,ALBANY,NY,12203,None,None,"206 QUAIL ST\nALBANY, NY 12203"
3,Albany,C,2019-08-08,EL CAMBIO MINI MARKET INC,EL CAMBIO MINI MARKET,32 CENTRAL AVENUE,ALBANY,NY,12210,10A,- exterior exit door in front of building in Retail area is held open continuously and is not properly screened.,"32 CENTRAL AVENUE\nALBANY, NY 12210"
4,Albany,A,2019-07-01,FAMILY DOLLAR STORES OF NY INC,FAMILY DOLLAR #2164,103 SARATOGA ST,COHOES,NY,12047,None,None,"103 SARATOGA ST\nCOHOES, NY 12047"
...,...,...,...,...,...,...,...,...,...,...,...,...
93270,Westchester,C,2019-04-12,AVENTURA DELI & GROCERY INC,AVENTURA DELI & GROCERY,322 WEBSTER AVENUE,NEW ROCHELLE,NY,10801,15D,- Thermometer is not provided to monitor internal temperatures of hot foods.,"322 WEBSTER AVENUE\nNEW ROCHELLE, NY 10801"
93271,Westchester,C,2019-06-13,L&L NEW YORK FOOD CORP,NSA SUPERMARKET,230 S BROADWAY,YONKERS,NY,10705,15D,- Thermometer is not provided in walk in cooler for eggs and Produce in the basement.,"230 S BROADWAY\nYONKERS, NY 10705"
93272,Westchester,B,2020-01-30,STOP&SHOP SUPERMARKET CO LLC THE,STOP&SHOP SPRMRKT 592,240 SANDFORD BLVD,MOUNT VERNON,NY,10550,12A,"- seafood salad and Kale Superfood salad in a Deli display cooler in the Deli prep area is observed uncovered and in close proximity to containers in a manner which threatens allergen cross contact with the other Cold Deli salads. the seafood salad (containing shellfish allergen) and Kale Superfood salad containing walnuts (tree nut allergen) were re-positioned in the Deli display cooler to prevent allergen cross contact during inspection. - cherry almond Muffins, pistachio Muffins and bana...","240 SANDFORD BLVD\nMOUNT VERNON, NY 10550"
93273,Warren,A,2019-12-31,STEWARTS SHOPS CORP,STEWARTS SHOP 380,360 GLEN ST,GLENS FALLS,NY,12801,None,None,"360 GLEN ST\nGLENS FALLS, NY 12801"


## Data Set Cleaning and Setup

After reviewing the data set, I created multiple functions to clean the data and imported it into another new table in my database.

In [ ]:
allInfoDFCleaned = allInfoDF
def fillEmptyTradeNames(dataFrame, tradeNameColumn, ownerNameColumn):
    '''
    When a store has no trade name, this function will fill the trade name with the store's owner name
    '''
    dataFrame[tradeNameColumn].fillna(dataFrame[ownerNameColumn], inplace=True)
    return dataFrame
fillEmptyTradeNames(allInfoDFCleaned, 'trade_name', 'owner_name')

def createInpectionID(dataFrame, tradeNameColumn, inspectionDateColumn):
    '''
    This function adds a column to the beginning of the data frame and constructs an
    inspectionID for every row. This is not a primary key since the values are not unique.
    '''
    tradeName = dataFrame[tradeNameColumn].astype(str)
    inspectionDate = dataFrame[inspectionDateColumn].astype(str)
    inspectionID = inspectionDate + '_' + tradeName
    dataFrame.insert(0, 'inspectionID', inspectionID)
    # Replaces all the spaces in the trade name portion of inspectionID with underscores
    count = 0
    while count < len(dataFrame):
        inspectionIDLocation = dataFrame.loc[count, 'inspectionID']
        inspectionIDFixed = inspectionIDLocation.replace(' ', '_')
        dataFrame.loc[count, 'inspectionID'] = inspectionIDFixed
        count += 1
    return dataFrame
createInpectionID(allInfoDFCleaned, 'trade_name', 'inspection_date')

def fillNullLocations(dataFrame, locColumn, streetColumn, cityColumn, stateColumn, zipColumn):
    '''
    This fills locations that are empty with a string made up of street, city, state, and zip columns
    '''
    count = 0
    while count < len(dataFrame):
        if dataFrame.loc[count, locColumn] == '':
            street = dataFrame.loc[count, streetColumn]
            city = dataFrame.loc[count, cityColumn]
            state = dataFrame.loc[count, stateColumn]
            zipCode = dataFrame.loc[count, zipColumn]
            address = street + '\n' + city + ', ' + state + ' ' + str(zipCode)
            dataFrame.loc[count, locColumn] = address
            count += 1
        else:
            count += 1
    return dataFrame
fillNullLocations(allInfoDFCleaned, 'location_1', 'street', 'city', 'state_code', 'zip_code')

def properCase(dataFrame, column):
    '''
    This corrects the selected column and converts the string to proper case i.e. city
    '''
    count = 0;
    while count < len(dataFrame):
        cellSelected = dataFrame.loc[count, column]
        properCaseCell = cellSelected.title()
        dataFrame.loc[count, column] = properCaseCell
        count += 1
properCase(allInfoDFCleaned, 'city')
allInfoDFCleaned

,inspectionID,county,inspection,inspection_date,owner_name,trade_name,street,city,state_code,zip_code,deficiency_number,deficiency_description,location_1
0,2019-08-21_ALBANY_FAST_FOOD,Albany,C,2019-08-21,ALBANY FAST FOOD INC,ALBANY FAST FOOD,9 NEW SCOTLAND AVE,Albany,NY,12208,02A,"- 15.46 pounds of bagged chips, bagged flour, and packages of noodles stored on shelves in retail area found to be rodent defiled with gnaw marks. Products destroyed under signed waiver during inspection.","9 NEW SCOTLAND AVE\nALBANY, NY 12208"
1,2019-10-09_ALL_STAR_PIZZA_&_DELI,Albany,A,2019-10-09,ALL STAR PIZZA & DELI INC,ALL STAR PIZZA & DELI,224 QUAIL STREET,Albany,NY,12203,None,None,"224 QUAIL STREET\nALBANY, NY 12203"
2,2019-09-06_EASHA_MART,Albany,A,2019-09-06,EASHA INC,EASHA MART,206 QUAIL ST,Albany,NY,12203,None,None,"206 QUAIL ST\nALBANY, NY 12203"
3,2019-08-08_EL_CAMBIO_MINI_MARKET,Albany,C,2019-08-08,EL CAMBIO MINI MARKET INC,EL CAMBIO MINI MARKET,32 CENTRAL AVENUE,Albany,NY,12210,10A,- exterior exit door in front of building in Retail area is held open continuously and is not properly screened.,"32 CENTRAL AVENUE\nALBANY, NY 12210"
4,2019-07-01_FAMILY_DOLLAR_#2164,Albany,A,2019-07-01,FAMILY DOLLAR STORES OF NY INC,FAMILY DOLLAR #2164,103 SARATOGA ST,Cohoes,NY,12047,None,None,"103 SARATOGA ST\nCOHOES, NY 12047"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93270,2019-04-12_AVENTURA_DELI_&_GROCERY,Westchester,C,2019-04-12,AVENTURA DELI & GROCERY INC,AVENTURA DELI & GROCERY,322 WEBSTER AVENUE,New Rochelle,NY,10801,15D,- Thermometer is not provided to monitor internal temperatures of hot foods.,"322 WEBSTER AVENUE\nNEW ROCHELLE, NY 10801"
93271,2019-06-13_NSA_SUPERMARKET,Westchester,C,2019-06-13,L&L NEW YORK FOOD CORP,NSA SUPERMARKET,230 S BROADWAY,Yonkers,NY,10705,15D,- Thermometer is not provided in walk in cooler for eggs and Produce in the basement.,"230 S BROADWAY\nYONKERS, NY 10705"
93272,2020-01-30_STOP&SHOP_SPRMRKT_592,Westchester,B,2020-01-30,STOP&SHOP SUPERMARKET CO LLC THE,STOP&SHOP SPRMRKT 592,240 SANDFORD BLVD,Mount Vernon,NY,10550,12A,"- seafood salad and Kale Superfood salad in a Deli display cooler in the Deli prep area is observed uncovered and in close proximity to containers in a manner which threatens allergen cross contact with the other Cold Deli salads. the seafood salad (containing shellfish allergen) and Kale Superfood salad containing walnuts (tree nut allergen) were re-positioned in the Deli display cooler to prevent allergen cross contact during inspection. - cherry almond Muffins, pistachio Muffins and bana...","240 SANDFORD BLVD\nMOUNT VERNON, NY 10550"
93273,2019-12-31_STEWARTS_SHOP_380,Warren,A,2019-12-31,STEWARTS SHOPS CORP,STEWARTS SHOP 380,360 GLEN ST,Glens Falls,NY,12801,None,None,"360 GLEN ST\nGLENS FALLS, NY 12801"


Personally, the data still felt a little incomplete. Unless you know New York's Sanitary Inspection Guidelines by heart, you don't know what each deficiency number specifically means or if it's a critical deficiency or not. After doing some research, I found this [pdf](https://agriculture.ny.gov/system/files/documents/2020/03/sanitary_inspection_guidelines_fsi-890.pdf) online that gave further explanation into what each deficiency number meant. I wanted to incorprate it with the existing data set so I manually input the values from the pdf into Excel, saved the workbook as a CSV file, and imported it into my personal database.

In [ ]:
deficiencyDescriptionQuery = '''
    SELECT *
    FROM deficiency_description;
'''
deficiencyDescriptionDF = pd.read_sql_query(deficiencyDescriptionQuery, con = connection)
deficiencyDescriptionDF

,deficiency_number,general_description,critical_deficiency,group_number,group_category
0,01A,*01A - Unpasteurized milk/milk products used,1,1,Food Received From Unapproved Sources
1,01B,*01B - Water or ice is not potable,1,1,Food Received From Unapproved Sources
2,01C,01C - Foods or ingredients from unapproved source,0,1,Food Received From Unapproved Sources
3,01D,"01D - Meat, poultry, or game products are not officially inspected plants",0,1,Food Received From Unapproved Sources
4,01E,*01E - Shellfish from unapproved source,1,1,Food Received From Unapproved Sources
...,...,...,...,...,...
72,16A,"16A - Chemicals or pesticides improperly labeled, stored or handled",0,16,Other Sanitation
73,16B,16B - Outside premises or loading zones improperly maintained,0,16,Other Sanitation
74,16C,16C - Vehicles are not clean or in good repair,0,16,Other Sanitation
75,16D,16D - Morgue area or bottle return area improperly maintained,0,16,Other Sanitation


I uploaded CSV files containing allInfoDFCleaned, [here](https://github.com/andrewcsmith1997/retailFoodStoreCountyAnalysis/blob/master/CSV%20Files/retail_food_store_inspections_andrew_smith.csv), and deficiencyDescriptionDF, [here](https://github.com/andrewcsmith1997/retailFoodStoreCountyAnalysis/blob/master/CSV%20Files/DeficiencyDescription.csv). 

## Two Sample T-Test Analysis

Now that I had set up my database to have all the information I wanted on each deficiency and inspection, I could pull the specific data I needed to start the two sample t-test process.

First, I needed to create a dataframe that pulled the relevant columns for the t-test.

In [ ]:
# Creates Data Frame from SQL Query
inspectionIDGroupByQuery = '''
    SELECT DISTINCT inspectionID, county, SUM(critical_deficiency) AS criticalDeficiencies
    FROM retail_food_store_inspections_andrew_smith LEFT OUTER JOIN deficiency_description
    ON retail_food_store_inspections_andrew_smith.deficiency_number = deficiency_description.deficiency_number
    GROUP BY inspectionID, county
    ORDER BY county;
'''
inspectionIDGroupByDF = pd.read_sql_query(inspectionIDGroupByQuery, con = connection)
inspectionIDGroupByDF['criticalDeficiencies'] = inspectionIDGroupByDF['criticalDeficiencies'].replace(np.nan, 0)
inspectionIDGroupByDF

,inspectionID,county,criticalDeficiencies
0,2019-08-21_ALBANY_FAST_FOOD,Albany,0.0
1,2019-10-16_BEVERAGE&SMOKE_CONVENIE,Albany,0.0
2,2019-10-09_ALL_STAR_PIZZA_&_DELI,Albany,0.0
3,2020-02-07_FALVOS_MEAT_CO,Albany,0.0
4,2019-09-06_EASHA_MART,Albany,0.0
...,...,...,...
25175,2019-08-23_KEUKA_LAKE_COFFEE_RSTRS,Yates,0.0
25176,2019-06-27_BURKHOLDER'S_BAKERY,Yates,0.0
25177,2019-03-22_BYRNE_DAIRY_STORE_58,Yates,0.0
25178,2019-05-15_SHIRKS_MEATS,Yates,0.0


Next, I needed to create a second dataframe that re-grouped the data by county. I could've done this with either a Python function or a SQL query.

In [ ]:
# Creates Data Frame based off previous Data Frame, Python Function Equivalent to countyGroupByQuery
def createNewDataFrame(dataFrame, groupColumnTitle, critDef):
    '''
    :param dataFrame: the original data frame that we are basing this new data frame off of
    :param groupColumnTitle: the title of the column we are choosing to group by
    :param critDef: the title of the critical deficiencies column from the original data frame
    :return: data frame consisting of total inspection, total critical deficiency, 
            and mean values determined by the column (groupColumnTitle) we 
            decided to group the data by
    '''
    # Creates the new dataframe that's going to be filled
    newDF = pd.DataFrame(columns = ['groupByColumn', 'totalInspections', 
                                    'totalCriticalDeficiencies', 'mean'])
    # Fills the groupByColumn with distinct values of the column entered by user
    newDF['groupByColumn'] = dataFrame[groupColumnTitle].unique()
    # Calculates inspection count, critical deficiencies sum, and critical deficiencies per inspection
    count = 0
    while count < len(newDF):
        counter = 0
        inspectionCount = 0
        critDefCount = 0
        while counter < len(dataFrame):
            if newDF.loc[count, 'groupByColumn'] == dataFrame.loc[counter, groupColumnTitle]:
                inspectionCount += 1
                critDefCount += dataFrame.loc[counter, critDef]
                counter += 1
            else:
                counter += 1
        newDF.loc[count, 'totalInspections'] = inspectionCount
        newDF.loc[count, 'totalCriticalDeficiencies'] = critDefCount
        newDF['mean'] = newDF['totalCriticalDeficiencies'] / newDF['totalInspections']
        count += 1
    return newDF
countyGroupByDF = createNewDataFrame(inspectionIDGroupByDF, 'county', 'criticalDeficiencies')
countyGroupByDF

,groupByColumn,totalInspections,totalCriticalDeficiencies,mean
0,Albany,439,38,0.0865604
1,Allegany,83,11,0.13253
2,Bronx,2386,493,0.206622
3,Broome,277,70,0.252708
4,Cattaraugus,96,8,0.0833333
...,...,...,...,...
57,Washington,77,11,0.142857
58,Wayne,96,3,0.03125
59,Westchester,968,242,0.25
60,Wyoming,54,3,0.0555556


In [ ]:
# Creates Data Frame from SQL Query, SQL Query Equivalent to createNewDataFrame
countyGroupByQuery = '''
    SELECT county, COUNT(DISTINCT inspectionID) AS inspections, SUM(critical_deficiency) AS criticalDeficiencies, (SUM(critical_deficiency)/COUNT(DISTINCT inspectionID)) AS average
    FROM retail_food_store_inspections_andrew_smith LEFT OUTER JOIN deficiency_description
    ON retail_food_store_inspections_andrew_smith.deficiency_number = deficiency_description.deficiency_number
    GROUP BY county;
'''
countyGroupByDFV2 = pd.read_sql_query(countyGroupByQuery, con = connection)
countyGroupByDFV2

,county,inspections,criticalDeficiencies,average
0,Albany,439,38.0,0.0866
1,Allegany,83,11.0,0.1325
2,Bronx,2386,493.0,0.2066
3,Broome,277,70.0,0.2527
4,Cattaraugus,96,8.0,0.0833
...,...,...,...,...
57,Washington,77,11.0,0.1429
58,Wayne,96,3.0,0.0313
59,Westchester,968,242.0,0.2500
60,Wyoming,54,3.0,0.0556


Then from those two dataframes (inspectionIDGroupByDF and countyGroupByDF), I calculated all of the necessary statistics to perform <br>the t-tests.

For the two sample t-test, I compared the mean of a county (e.g., Albany) to the mean of every other county combined.
>mean = (total critical deficiencies found) / (# of inspections)

My hypotheses for the tests were:
>Null: (County Mean) - (Other Counties Mean) = 0 <br>
>Alt: (County Mean) - (Other Counties Mean) ≠ 0

In [ ]:
def tTestAnalysisNewDataFrame(newDF, observationDF, observationGroupBy, observationCritDef, newGroupBy, inspectionTotal,
                         avg, confidence):
    '''
    :param newDF: the data frame where the data is grouped by a specific column, (e.g., county)
    :param observationDF: the original data frame that isn't grouped by the specific column
    :param observationGroupBy: the title of the column in the observationDF that we're choosing to group by, (e.g., county)
    :param observationCritDef: the title of the critical deficiencies column from observationDF
    :param newGroupBy: the title of the column in the newDF that we're choosing to group by, (e.g., county)
    :param inspectionTotal: title of inspection count column from newDF
    :param avg: title of the column that contains the first mean we want to compare, from newDF
    :param confidence: the confidence interval we want to run the test on
    :return: data frame consisting of statistics necessary to compute two sample t-tests and the results of those
            tests for every record the newDF is grouped by
    '''
    # Calculates the count of inspections in other counties, count of critical deficiencies in other counties,
    # variance of each county, and variance of every other county grouped together.
    analysisNewDF = newDF
    analysisNewDF['diffCountyInspections'] = len(observationDF) - newDF[inspectionTotal]
    analysisNewDF['diffCountyTotalCritDef'] = ''
    analysisNewDF['diffCountyMean'] = ''
    analysisNewDF['variance'] = ''
    analysisNewDF['diffCountyVariance'] = ''
    count = 0
    while count < len(newDF):
        counter = 0
        diffCountyCritDefCount = 0
        variance = 0
        diffCountyVariance = 0
        while counter < len(observationDF):
            if newDF.loc[count, newGroupBy] == observationDF.loc[counter, observationGroupBy]:
                variance += ((observationDF.loc[counter, observationCritDef] - newDF.loc[count, avg])**2)
                counter += 1
            else:
                diffCountyCritDefCount += observationDF.loc[counter, observationCritDef]
                diffCountyVariance += ((observationDF.loc[counter, observationCritDef] - newDF.loc[count, avg]) ** 2)
                counter += 1
        analysisNewDF.loc[count, 'diffCountyTotalCritDef'] = diffCountyCritDefCount
        analysisNewDF.loc[count, 'diffCountyMean'] = (diffCountyCritDefCount / analysisNewDF.loc[count, 'diffCountyInspections'])
        analysisNewDF.loc[count, 'variance'] = variance / (newDF.loc[count, inspectionTotal] - 1)
        analysisNewDF.loc[count, 'diffCountyVariance'] = diffCountyVariance / \
                                                         (analysisNewDF.loc[count, 'diffCountyInspections'] - 1)
        count += 1
    # Calculates the difference between the two means, the standard error of difference, the degrees of freedom,
    # and the test statistic.
    analysisNewDF['meanDiff'] = ''
    analysisNewDF['standardErrDiff'] = ''
    analysisNewDF['dF'] = ''
    analysisNewDF['meanDiff'] = newDF[avg] - analysisNewDF['diffCountyMean']
    analysisNewDF['standardErrDiff'] = (((analysisNewDF['variance'] / newDF[inspectionTotal]) +
                                        (analysisNewDF['diffCountyVariance'] / (len(observationDF) -
                                        newDF[inspectionTotal]))) ** 0.5)
    analysisNewDF['dF'] = ((((analysisNewDF['variance'] / newDF[inspectionTotal]) +
                           (analysisNewDF['diffCountyVariance'] / analysisNewDF['diffCountyInspections'])) ** 2) / \
                           ((((analysisNewDF['variance'] / newDF[inspectionTotal]) ** 2) / \
                            (newDF[inspectionTotal] - 1)) +
                           (((analysisNewDF['diffCountyVariance'] / analysisNewDF['diffCountyInspections']) ** 2) / \
                            (analysisNewDF['diffCountyVariance']))))
    analysisNewDF['dF'] = analysisNewDF['dF'].astype(float)
    analysisNewDF = analysisNewDF.round({'dF': 0})
    analysisNewDF['dF'] = analysisNewDF['dF'].astype(int)
    analysisNewDF['t-Stat'] = analysisNewDF['meanDiff'] / (((analysisNewDF['variance'] / newDF[inspectionTotal]) +
                                                           (analysisNewDF['diffCountyVariance'] / \
                                                            analysisNewDF['diffCountyInspections'])) ** 0.5)
    # Calculates the two tail p-value, critical value, margin of error, confidence interval, and determines if we reject or
    # fail to reject the null hypothesis.
    analysisNewDF['pValue'] = ''
    analysisNewDF['criticalValue'] = ''
    analysisNewDF['marginOfError'] = ''
    analysisNewDF['lowerLimit'] = ''
    analysisNewDF['upperLimit'] = ''
    analysisNewDF['verdict'] = ''
    analysisNewDF['statSignificance'] = ''
    count = 0
    while count < len(newDF):
        pValCalcDF = observationDF
        pValCalcDFSame = pValCalcDF[pValCalcDF[observationGroupBy] == newDF.loc[count, newGroupBy]]
        pValCalcDFDiff = pValCalcDF[pValCalcDF[observationGroupBy] != newDF.loc[count, newGroupBy]]
        analysisNewDF.loc[count, 'pValue'] = ttest_ind(pValCalcDFSame[observationCritDef],
                                                         pValCalcDFDiff[observationCritDef], equal_var = False)[1]
        analysisNewDF.loc[count, 'criticalValue'] = norm.ppf((1 + confidence) / 2)
        analysisNewDF.loc[count, 'marginOfError'] = analysisNewDF.loc[count, 'standardErrDiff'] * \
                                                         analysisNewDF.loc[count, 'criticalValue']
        analysisNewDF.loc[count, 'lowerLimit'] = analysisNewDF.loc[count, 'meanDiff'] - analysisNewDF.loc[count, 'marginOfError']
        analysisNewDF.loc[count, 'upperLimit'] = analysisNewDF.loc[count, 'meanDiff'] + analysisNewDF.loc[count, 'marginOfError']

        if analysisNewDF.loc[count, 't-Stat'] > analysisNewDF.loc[count, 'criticalValue']:
            analysisNewDF.loc[count, 'verdict'] = 'We reject the null that there is no difference between the sample means.'
            analysisNewDF.loc[count, 'statSignificance'] = 'The difference is statistically significant at the <' + str(round((1 - confidence), 2)) + \
                                                ' level. With ' + str(int(confidence * 100)) + '% confidence,' \
                                                ' the difference in means is between ' + str(round(analysisNewDF.loc[count, 'lowerLimit'], 4)) + \
                                                ' and ' + str(round(analysisNewDF.loc[count, 'upperLimit'], 4)) + '.'
        elif analysisNewDF.loc[count, 't-Stat'] < ((analysisNewDF.loc[count, 'criticalValue']) * -1):
            analysisNewDF.loc[count, 'verdict'] = 'We reject the null that there is no difference between the sample means.'
            analysisNewDF.loc[count, 'statSignificance'] = 'The difference is statistically significant at the <' + str(round((1 - confidence), 2)) + \
                                                ' level. With ' + str(int(confidence * 100)) + '% confidence,' \
                                                ' the difference in means is between ' + str(round(analysisNewDF.loc[count, 'lowerLimit'], 4)) + \
                                                ' and ' + str(round(analysisNewDF.loc[count, 'upperLimit'], 4)) + '.'
        else:
            analysisNewDF.loc[count, 'verdict'] = 'We fail to reject the null that there is no difference between the sample means.'
            analysisNewDF.loc[count, 'statSignificance'] = 'The difference is not statistically significant at the ' + \
                                                str(round(analysisNewDF.loc[count, 'pValue'], 4)) + \
                                                ' level. With ' + str(int(confidence * 100)) + '% confidence,' \
                                                ' the difference in means is between ' + str(round(analysisNewDF.loc[count, 'lowerLimit'], 4)) + \
                                                ' and ' + str(round(analysisNewDF.loc[count, 'upperLimit'], 4)) + '.'
        count += 1
    return analysisNewDF
tTestAnalysisNewDF = tTestAnalysisNewDataFrame(countyGroupByDF, inspectionIDGroupByDF, 'county', 'criticalDeficiencies', 'groupByColumn',
                                     'totalInspections','mean', 0.95)
tTestAnalysisNewDF

,groupByColumn,totalInspections,totalCriticalDeficiencies,mean,diffCountyInspections,diffCountyTotalCritDef,diffCountyMean,variance,diffCountyVariance,meanDiff,standardErrDiff,dF,t-Stat,pValue,criticalValue,marginOfError,lowerLimit,upperLimit,verdict,statSignificance
0,Albany,439,38,0.0865604,24741,4339,0.175377,0.0883806,0.249523,-0.0888165,0.0145399,89,-6.10848,2.02078e-09,1.95996,0.0284976,-0.117314,-0.0603189,We reject the null that there is no difference between the sample means.,"The difference is statistically significant at the <0.05 level. With 95% confidence, the difference in means is between -0.1173 and -0.0603."
1,Allegany,83,11,0.13253,25097,4366,0.173965,0.116368,0.241216,-0.0414349,0.0375717,82,-1.10282,0.273276,1.95996,0.0736392,-0.115074,0.0322043,We fail to reject the null that there is no difference between the sample means.,"The difference is not statistically significant at the 0.2733 level. With 95% confidence, the difference in means is between -0.1151 and 0.0322."
2,Bronx,2386,493,0.206622,22794,3884,0.170396,0.284753,0.235515,0.0362262,0.0113875,37,3.18122,0.00147892,1.95996,0.0223191,0.0139071,0.0585454,We reject the null that there is no difference between the sample means.,"The difference is statistically significant at the <0.05 level. With 95% confidence, the difference in means is between 0.0139 and 0.0585."
3,Broome,277,70,0.252708,24903,4307,0.172951,0.341705,0.244258,0.0797565,0.0352619,262,2.26184,0.0244591,1.95996,0.069112,0.0106445,0.148869,We reject the null that there is no difference between the sample means.,"The difference is statistically significant at the <0.05 level. With 95% confidence, the difference in means is between 0.0106 and 0.1489."
4,Cattaraugus,96,8,0.0833333,25084,4369,0.174175,0.0982456,0.247858,-0.0908414,0.0321446,94,-2.82603,0.00571815,1.95996,0.0630022,-0.153844,-0.0278393,We reject the null that there is no difference between the sample means.,"The difference is statistically significant at the <0.05 level. With 95% confidence, the difference in means is between -0.1538 and -0.0278."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,Washington,77,11,0.142857,25103,4366,0.173923,0.203008,0.240175,-0.0310663,0.0514396,76,-0.603938,0.547668,1.95996,0.10082,-0.131886,0.0697534,We fail to reject the null that there is no difference between the sample means.,"The difference is not statistically significant at the 0.5477 level. With 95% confidence, the difference in means is between -0.1319 and 0.0698."
58,Wayne,96,3,0.03125,25084,4374,0.174374,0.0516447,0.260221,-0.143124,0.0234167,87,-6.11206,1.94157e-08,1.95996,0.0458958,-0.18902,-0.0972283,We reject the null that there is no difference between the sample means.,"The difference is statistically significant at the <0.05 level. With 95% confidence, the difference in means is between -0.189 and -0.0972."
59,Westchester,968,242,0.25,24212,4135,0.170783,0.557911,0.232405,0.0792169,0.0242065,464,3.27255,0.00109956,1.95996,0.0474438,0.0317731,0.126661,We reject the null that there is no difference between the sample means.,"The difference is statistically significant at the <0.05 level. With 95% confidence, the difference in means is between 0.0318 and 0.1267."
60,Wyoming,54,3,0.0555556,25126,4374,0.174083,0.0534591,0.253514,-0.118527,0.0316239,53,-3.74802,0.000433934,1.95996,0.0619818,-0.180509,-0.0565453,We reject the null that there is no difference between the sample means.,"The difference is statistically significant at the <0.05 level. With 95% confidence, the difference in means is between -0.1805 and -0.0565."


I uploaded a CSV file containing tTestAnalysisNewDF, [here](https://github.com/andrewcsmith1997/retailFoodStoreCountyAnalysis/blob/master/CSV%20Files/retailFoodStoreCountyAnalysis.csv).

## Conclusion

Null: (County Mean) - (Other Counties Mean) = 0 <br>
Alt: (County Mean) - (Other Counties Mean) ≠ 0

In [ ]:
tTestAnalysisNewDF[['groupByColumn', 'meanDiff', 'pValue', 'verdict', 'statSignificance']]

,groupByColumn,meanDiff,pValue,verdict,statSignificance
0,Albany,-0.0888165,2.02078e-09,We reject the null that there is no difference between the sample means.,"The difference is statistically significant at the <0.05 level. With 95% confidence, the difference in means is between -0.1173 and -0.0603."
1,Allegany,-0.0414349,0.273276,We fail to reject the null that there is no difference between the sample means.,"The difference is not statistically significant at the 0.2733 level. With 95% confidence, the difference in means is between -0.1151 and 0.0322."
2,Bronx,0.0362262,0.00147892,We reject the null that there is no difference between the sample means.,"The difference is statistically significant at the <0.05 level. With 95% confidence, the difference in means is between 0.0139 and 0.0585."
3,Broome,0.0797565,0.0244591,We reject the null that there is no difference between the sample means.,"The difference is statistically significant at the <0.05 level. With 95% confidence, the difference in means is between 0.0106 and 0.1489."
4,Cattaraugus,-0.0908414,0.00571815,We reject the null that there is no difference between the sample means.,"The difference is statistically significant at the <0.05 level. With 95% confidence, the difference in means is between -0.1538 and -0.0278."
...,...,...,...,...,...
57,Washington,-0.0310663,0.547668,We fail to reject the null that there is no difference between the sample means.,"The difference is not statistically significant at the 0.5477 level. With 95% confidence, the difference in means is between -0.1319 and 0.0698."
58,Wayne,-0.143124,1.94157e-08,We reject the null that there is no difference between the sample means.,"The difference is statistically significant at the <0.05 level. With 95% confidence, the difference in means is between -0.189 and -0.0972."
59,Westchester,0.0792169,0.00109956,We reject the null that there is no difference between the sample means.,"The difference is statistically significant at the <0.05 level. With 95% confidence, the difference in means is between 0.0318 and 0.1267."
60,Wyoming,-0.118527,0.000433934,We reject the null that there is no difference between the sample means.,"The difference is statistically significant at the <0.05 level. With 95% confidence, the difference in means is between -0.1805 and -0.0565."


When the null is rejected, the difference between the means is correlated to the county used to group the samples.

So, is there a correlation between County and Number of Critical Deficiencies Found per Sanitary Inspection? 
>In the counties where we rejected the null, yes.


## Recommendation

For a visual of which counties have an above or below average number of critical deficiencies found per inspection, check out this [dashboard](https://public.tableau.com/profile/andrew.smith7698#!/vizhome/NYRetailFoodStoreCountyAnalysis/CountyAnalysisDashBoard) I made on Tableau.

![picture](https://github.com/andrewcsmith1997/retailFoodStoreCountyAnalysis/blob/master/Images/nyRetailFoodStoreCountyAnalysisDashBoard.PNG)

This data is useful for a number of audiences.

For the general consumer of retail food stores:
>I recommend using this data to find which counties on average have fewer critical deficiencies found per inspection and to get food from those counties if possible.

For the New York State Department of Agriculture and Markets, Division of Food Safety and Inspection: 
>I recommend using this data to spark further research into finding the reasons why certain counties have an above or below average number of critical deficiencies found per inspection.

For current/potential retail food store owners: 
>I recommend using this data to understand which counties would be better (sanitary wise) when locating/re-locating a store.